# Basic Authentication Demo

This notebook demonstrates running the ETL pipeline against an API endpoint that requires HTTP Basic Authentication.

## Prerequisites

Start the mock API service:
```bash
make up-keycloak
```

The mock API will be available at `http://mock-api:8000` from within the Docker network.

## Credentials

- Username: `test-user`
- Password: `test-password`

In [2]:
import sys
from pathlib import Path

In [3]:
project_root = Path("/opt/spark/app")
src_path = project_root / "src"

if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import sha2, expr
from pipeline.orchestrator import run_pipeline

In [5]:
spark = (
    SparkSession.builder
    .appName("basic_auth_demo_pipeline")
    .getOrCreate()
)

## Create Source DataFrame

Generate a DataFrame with unique tracking IDs that will be used to make API requests.

In [5]:
df = (
    spark.range(50)
         .repartition(4)
         .select(
             sha2(expr("uuid()"), 256).alias("tracking_id")
         )
)
df.show(10)

+--------------------+
|         tracking_id|
+--------------------+
|ec50e4088521534de...|
|ae348af5d0b858d87...|
|26614713adba488b1...|
|ce828ee9df8adf1ba...|
|4424b67b80cd40e9e...|
|5471cf7fe7b51978a...|
|8b4f7dc1ebe567a08...|
|75d2fe05c76a440f4...|
|5b442e02974d86ca8...|
|1b32b13767ce237be...|
+--------------------+
only showing top 10 rows



## Run Pipeline

Execute the ETL pipeline using the basic auth configuration.

The pipeline will automatically add the `Authorization: Basic <base64>` header to all requests.

In [6]:
config_path = project_root / "configs" / "examples" / "basic_auth_demo.yml"

In [7]:
run_pipeline(
    spark=spark,
    config_path=config_path,
    source_df=df,
    source_id="tracking_id"
)

2026-02-08 14:13:33,612 [INFO] [PipelineOrchestrator]: Authentication does not have a runtime service... skipping
2026-02-08 14:13:33,612 [INFO] [PipelineOrchestrator]: Adding authentication middleware
2026-02-08 14:13:33,613 [INFO] [PipelineOrchestrator]: ====== Executing Pipeline ======
2026-02-08 14:13:33,613 [INFO] [PipelineOrchestrator]: Request from URL: /http://mock-api:8000/api/basic/data
2026-02-08 14:13:33,614 [INFO] [TableManager]: Creating database demo
2026-02-08 14:13:34,096 [WARNING] [TableManager]: Sink table OVERWRITE enabled. Table basic_auth_demo_response will be replaced.
2026-02-08 14:13:41,006 [INFO] [TableManager]: Created Delta table: demo.basic_auth_demo_response
2026-02-08 14:13:43,090 [INFO] [BatchProcessor]: ➤ Attempt 1: Processing 1 batches
2026-02-08 14:13:43,090 [INFO] [BatchProcessor]:     → Processing batch 1/1
2026-02-08 14:13:44,024 [INFO] numexpr.utils: Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2

## Verify Results

Read the sink table to verify the API responses were captured.

In [6]:
response_df = spark.table("demo.basic_auth_demo_response")
response_df.show(10)

+--------------------+--------------------+--------------------+------+--------------------+--------------+--------------------+-----------+--------------------+--------------------+-------+-------------+--------+--------------------+--------------------+
|          request_id|            row_hash|                 url|method|     request_headers|request_params|    request_metadata|status_code|    response_headers|           body_text|success|error_message|attempts|   response_metadata|       _request_time|
+--------------------+--------------------+--------------------+------+--------------------+--------------+--------------------+-----------+--------------------+--------------------+-------+-------------+--------+--------------------+--------------------+
|93c671b14bac483d7...|959a987de214ba419...|/http://mock-api:...|   GET|{"Accept": "appli...|            {}|{"vendor": "mock-...|        200|{"Date": "Mon, 09...|{"request_id":"50...|   true|         NULL|       1|{"connection_warm..

In [7]:
# Summary statistics
print(f"Total records: {response_df.count()}")
response_df.groupBy("status_code").count().show()

Total records: 100
+-----------+-----+
|status_code|count|
+-----------+-----+
|        200|  100|
+-----------+-----+



In [10]:
# Sample response body - note the auth_method field shows "basic:test-user"
response_df.select("body_text").limit(3).show(truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|body_text                                                                                                                                                                                                                                                                                                                                                                                                                                                   |
+---------------------------------------------------------------------------------------------------------

----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 45050)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/conda/lib/python3.11/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/opt/conda/lib/python3.11/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/conda/lib/python3.11/socketserver.py", line 755, in __init__
    self.handle()
  File "/usr/local/spark/python/pyspark/accumulators.py", line 295, in handle
    poll(accum_updates)
  File "/usr/local/spark/python/pyspark/accumulators.py", line 267, in poll
    if self.rfile in r and func():
                           ^^^^^^
  File "/usr/local/spark/python/pyspark/accumulators.py", line 271, in accum_updates
    num_updates =